In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import re
import time
import tensorflow as tf

import platform
path = "c:/Windows/Fonts/malgun.ttf" 
from matplotlib import font_manager, rc 
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') 
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [2]:
import pickle
with open('./output_data/QuestionsForLabelClassifier.pickle', 'rb') as f:
    df_all_question = pickle.load(f)
df_all_question.head()

,question,mecab,tag_intent,tag_adjs,all_tags
0,남가좌2동 엘본헤어하우스 비용 알려주실래요,남가좌동 엘 본 헤어 하우스 비용 알려 주,price,[],[price]
1,맷블랙헤어성신여대점 장소 아냐 모르냐,맷 블랙 헤어 성 신여 대점 장소 아냐 모르 냐,location,[],[location]
2,진짜 최고의 남심 저격할만한 위생적인 투블럭괜찮은 곳 어디있죠,진짜 최고 남심 저격 할 만 위생 적 투 블럭 괜찮 곳 있 죠,recommend,"[satisfaction, satisfaction, sanitation]","[recommend, satisfaction, satisfaction, sanita..."
3,명일역 9hair 하는 가격 궁금하네요,명일역 가격 궁금,price,[],[price]
4,균형잡힌 럭셔리한 부해보이지 않는 알려주고 싶은 데 여쭤볼려고,균형 잡힌 럭셔리 부 보이 않 알려 주 싶 데 여쭤 볼려고,recommend,"[satisfaction, mood, satisfaction]","[recommend, satisfaction, mood, satisfaction]"


In [3]:
df_all_question['mecab'].isnull().sum()

0

In [4]:
mecab_sep = df_all_question.mecab.str.replace(' ',',')
mecab_sep

0                     남가좌동,엘,본,헤어,하우스,비용,알려,주
1                  맷,블랙,헤어,성,신여,대점,장소,아냐,모르,냐
2          진짜,최고,남심,저격,할,만,위생,적,투,블럭,괜찮,곳,있,죠
3                                   명일역,가격,궁금
4            균형,잡힌,럭셔리,부,보이,않,알려,주,싶,데,여쭤,볼려고
                          ...                
2125199                클레오파트라,미용실,요금,가르쳐,주,세요
2125200           뤠잇한,멀,않,스핀,스,왈,로펌,괜찮,미용실,어데
2125201           자연광,빛나,스핀,스,왈,로펌,추천,할,미용실,있
2125202        헤어,라인,잘,잡,주,포근,추천,가능,미용실,어데,인지
2125203         체인점,많,언밸런스,컷,추천,할,만,미용실,있,겠,죠
Name: mecab, Length: 2125204, dtype: object

In [5]:
mecab_sep=mecab_sep.str.split(',')
mecab_sep

0                           [남가좌동, 엘, 본, 헤어, 하우스, 비용, 알려, 주]
1                      [맷, 블랙, 헤어, 성, 신여, 대점, 장소, 아냐, 모르, 냐]
2          [진짜, 최고, 남심, 저격, 할, 만, 위생, 적, 투, 블럭, 괜찮, 곳, 있, 죠]
3                                              [명일역, 가격, 궁금]
4              [균형, 잡힌, 럭셔리, 부, 보이, 않, 알려, 주, 싶, 데, 여쭤, 볼려고]
                                 ...                        
2125199                        [클레오파트라, 미용실, 요금, 가르쳐, 주, 세요]
2125200               [뤠잇한, 멀, 않, 스핀, 스, 왈, 로펌, 괜찮, 미용실, 어데]
2125201               [자연광, 빛나, 스핀, 스, 왈, 로펌, 추천, 할, 미용실, 있]
2125202           [헤어, 라인, 잘, 잡, 주, 포근, 추천, 가능, 미용실, 어데, 인지]
2125203            [체인점, 많, 언밸런스, 컷, 추천, 할, 만, 미용실, 있, 겠, 죠]
Name: mecab, Length: 2125204, dtype: object

In [6]:
df_all_question['mecab_sep'] = mecab_sep

In [7]:
df_all_question

,question,mecab,tag_intent,tag_adjs,all_tags,mecab_sep
0,남가좌2동 엘본헤어하우스 비용 알려주실래요,남가좌동 엘 본 헤어 하우스 비용 알려 주,price,[],[price],"[남가좌동, 엘, 본, 헤어, 하우스, 비용, 알려, 주]"
1,맷블랙헤어성신여대점 장소 아냐 모르냐,맷 블랙 헤어 성 신여 대점 장소 아냐 모르 냐,location,[],[location],"[맷, 블랙, 헤어, 성, 신여, 대점, 장소, 아냐, 모르, 냐]"
2,진짜 최고의 남심 저격할만한 위생적인 투블럭괜찮은 곳 어디있죠,진짜 최고 남심 저격 할 만 위생 적 투 블럭 괜찮 곳 있 죠,recommend,"[satisfaction, satisfaction, sanitation]","[recommend, satisfaction, satisfaction, sanita...","[진짜, 최고, 남심, 저격, 할, 만, 위생, 적, 투, 블럭, 괜찮, 곳, 있, 죠]"
3,명일역 9hair 하는 가격 궁금하네요,명일역 가격 궁금,price,[],[price],"[명일역, 가격, 궁금]"
4,균형잡힌 럭셔리한 부해보이지 않는 알려주고 싶은 데 여쭤볼려고,균형 잡힌 럭셔리 부 보이 않 알려 주 싶 데 여쭤 볼려고,recommend,"[satisfaction, mood, satisfaction]","[recommend, satisfaction, mood, satisfaction]","[균형, 잡힌, 럭셔리, 부, 보이, 않, 알려, 주, 싶, 데, 여쭤, 볼려고]"
...,...,...,...,...,...,...
2125199,클레오파트라미용실 요금 가르쳐주세요,클레오파트라 미용실 요금 가르쳐 주 세요,price,[],[price],"[클레오파트라, 미용실, 요금, 가르쳐, 주, 세요]"
2125200,그뤠잇한 멀지 않은 스핀스왈로펌괜찮은 미용실 어데고,뤠잇한 멀 않 스핀 스 왈 로펌 괜찮 미용실 어데,recommend,"[satisfaction, location]","[recommend, satisfaction, location]","[뤠잇한, 멀, 않, 스핀, 스, 왈, 로펌, 괜찮, 미용실, 어데]"
2125201,자연광의 빛나는 스핀스왈로펌추천할 미용실 있는지,자연광 빛나 스핀 스 왈 로펌 추천 할 미용실 있,recommend,"[mood, sanitation]","[recommend, mood, sanitation]","[자연광, 빛나, 스핀, 스, 왈, 로펌, 추천, 할, 미용실, 있]"
2125202,헤어라인 잘 잡아주는 포근한 추천가능한 미용실 어데인지아나,헤어 라인 잘 잡 주 포근 추천 가능 미용실 어데 인지,recommend,"[satisfaction, mood]","[recommend, satisfaction, mood]","[헤어, 라인, 잘, 잡, 주, 포근, 추천, 가능, 미용실, 어데, 인지]"


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [9]:
def tokenize_text(x_train, x_test):
    t = Tokenizer(num_words=40000) #빈도가 1~40000개 에 해당하는 단어만 선택
    t.fit_on_texts(x_train)
    x_train = t.texts_to_sequences(x_train)
    x_test = t.texts_to_sequences(x_test)
    
    max_features = len(t.word_counts)
    X_train = pad_sequences(x_train, maxlen=100) #단어 수를 100개로 맞춤
    X_test = pad_sequences(x_test, maxlen=100) #만일  
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    
    return X_train, X_test

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X1_train, X1_test, y1_train, y1_test = \
    train_test_split(df_all_question['mecab_sep'].values
                    ,df_all_question['tag_intent'].values
                    ,test_size=0.1)

In [12]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(df_all_question['mecab_sep'].values
                    ,df_all_question['all_tags'].values
                    ,test_size=0.1)

In [13]:
%%time
X1_train, X1_test = tokenize_text(X1_train, X1_test)

Wall time: 25.2 s


In [14]:
%%time
X2_train, X2_test = tokenize_text(X2_train, X2_test)

Wall time: 24.6 s


In [15]:
X1_train.shape, X1_test.shape

((1912683, 100), (212521, 100))

In [16]:
X1_train, X1_test

(array([[  0.,   0.,   0., ...,  28.,   2.,  23.],
        [  0.,   0.,   0., ...,  28., 133., 234.],
        [  0.,   0.,   0., ...,   8.,   2.,  20.],
        ...,
        [  0.,   0.,   0., ...,  12.,   2.,  20.],
        [  0.,   0.,   0., ...,  13.,   6.,  20.],
        [  0.,   0.,   0., ...,  34.,  19.,  20.]], dtype=float32),
 array([[  0.,   0.,   0., ...,  65.,  43., 113.],
        [  0.,   0.,   0., ...,  72.,   7.,   2.],
        [  0.,   0.,   0., ...,  40.,  11.,  77.],
        ...,
        [  0.,   0.,   0., ...,   1.,   8.,  19.],
        [  0.,   0.,   0., ...,   4.,   1.,   8.],
        [  0.,   0.,   0., ...,  43., 215.,  36.]], dtype=float32))

In [17]:
X2_train.shape, X2_test.shape

((1912683, 100), (212521, 100))

In [18]:
X2_train, X2_test

(array([[  0.,   0.,   0., ...,  15.,   7.,   2.],
        [  0.,   0.,   0., ...,  52.,   4.,  59.],
        [  0.,   0.,   0., ...,  44.,  52.,  77.],
        ...,
        [  0.,   0.,   0., ...,   5.,  52., 101.],
        [  0.,   0.,   0., ...,  53.,  64.,   2.],
        [  0.,   0.,   0., ...,  40.,   2.,  23.]], dtype=float32),
 array([[  0.,   0.,   0., ...,  39.,  11.,  77.],
        [  0.,   0.,   0., ...,   2., 236.,  20.],
        [  0.,   0.,   0., ...,  39.,  52.,  77.],
        ...,
        [  0.,   0.,   0., ...,  44.,  11.,  77.],
        [  0.,   0.,   0., ...,  27.,   5., 224.],
        [  0.,   0.,   0., ...,   8.,   2.,  20.]], dtype=float32))

In [19]:
y1_train, y1_test # 벡터화 필요함.

(array(['location', 'location', 'recommend', ..., 'location', 'recommend',
        'review'], dtype=object),
 array(['review', 'recommend', 'location', ..., 'recommend', 'recommend',
        'review'], dtype=object))

In [20]:
# keras.utils.np_utils.to_categorical

In [21]:
def one_hot_encoding(y_train, y_test):
    t = Tokenizer(num_words=40000)
    t.fit_on_texts(y_train)
    y_train = t.texts_to_sequences(y_train)
    y_test = t.texts_to_sequences(y_test)
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    return y_train, y_test

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from keras.layers import Conv1D, MaxPool1D

Using TensorFlow backend.


In [28]:
model = Sequential()
model.add(Embedding(40000, 128))
model.add(Dropout(0.5))
# model.add(Dense(6, input_shape=(2,)))
model.add(Conv1D(64,5, padding='valid', activation='relu', strides=1))
model.add(MaxPool1D(pool_size=2))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
# model.add(Dense(1, activation='linear'))

In [29]:
model.compile(loss='categorical_crossentropy'
             , optimizer='adam'
             , metrics=['accuracy'])

In [30]:
y1_train

array(['location', 'location', 'recommend', ..., 'location', 'recommend',
       'review'], dtype=object)

In [31]:
y1_test

array(['review', 'recommend', 'location', ..., 'recommend', 'recommend',
       'review'], dtype=object)

In [33]:
y1_train, y1_test = one_hot_encoding(y1_train, y1_test)
y1_train, y1_test

(array([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], dtype=float32),
 array([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], dtype=float32))

In [34]:
y1_train, y1_test

(array([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], dtype=float32),
 array([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], dtype=float32))

In [35]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
hist = model.fit(X1_train, y1_train
                , batch_size=10
                , epochs=1
                , validation_data=(X1_test, y1_test)
                , callbacks=[early_stopping])


Train on 1912683 samples, validate on 212521 samples
Epoch 1/10
1912683/1912683 [==============================] - 15150s 8ms/step - loss: 0.0087 - accuracy: 0.9962 - val_loss: 0.0055 - val_accuracy: 0.9969
Epoch 2/10
   1810/1912683 [..............................] - ETA: 4:11:08 - loss: 0.0059 - accuracy: 0.9972

KeyboardInterrupt: 